### 가게별 UserID, ItemID, Rating, Timestamp 뽑기
- user_review_id = {}
- review_json = {}
- image_json = {}
- item_meta = {}

In [38]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

In [2]:
data = pd.read_csv("C:/Users/zhqhd/Desktop/BOAZ 공부 자료/ADV/data/Final Res.csv")
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,역,검색어
0,도전최강달인왕만두,한식,한식/백반/한정식,서울특별시 강동구 고덕로 333,5호선 상일동,373.362877,상일동,상일동역 도전최강달인왕만두
1,이때,한식,한식/백반/한정식,서울특별시 마포구 동교로38안길 7,경의중앙 홍대입구,360.468770,홍대입구,홍대입구역 이때
2,L.A.D,한식,한식/백반/한정식,서울특별시 마포구 잔다리로3안길 23,2호선 홍대입구,583.594176,홍대입구,홍대입구역 L.A.D
3,제이씨에스푸드,한식,한식/백반/한정식,서울특별시 관악구 신림로14길 3,2호선 봉천,860.540255,봉천,봉천역 제이씨에스푸드
4,비지트,한식,한식/백반/한정식,서울특별시 서초구 동광로18길 82,2호선 방배,810.975643,방배,방배역 비지트
...,...,...,...,...,...,...,...,...
135805,얼음과자,패스트푸드,아이스크림판매,서울특별시 강서구 금낭화로 135,5호선 방화,27.430119,방화,방화역 얼음과자
135806,오이소서울,패스트푸드,아이스크림판매,서울특별시 강동구 고덕로80길 123,5호선 상일동,983.009054,상일동,상일동역 오이소서울
135807,얼음왕국명일14,패스트푸드,아이스크림판매,서울특별시 강동구 상암로 214,5호선 굽은다리(강동구민회관앞),347.273163,굽은다리,굽은다리역 얼음왕국명일14
135808,얼음과자,패스트푸드,아이스크림판매,서울특별시 강동구 고덕로83길 6,5호선 상일동,1380.011573,상일동,상일동역 얼음과자


In [3]:
items = data["검색어"].head(20)
items

0     상일동역 도전최강달인왕만두
1           홍대입구역 이때
2        홍대입구역 L.A.D
3        봉천역 제이씨에스푸드
4            방배역 비지트
5        총신대입구역 쓰리에프
6             종각역 채움
7         선유도역 호구의주방
8             서초역 다향
9        압구정역 고향생막걸리
10      남부터미널역 든든한우家
11        신목동역 이삭토스트
12        아차산역 옥돌영양탕
13     몽촌토성역 양푼이김치찌개
14         애오개역 은진이네
15         오목교역 흥부보쌈
16         문정역 고수의족발
17     선정릉역 호랑이가만든족발
18      왕십리역 왕십리제일곱창
19           혜화역 커피빈
Name: 검색어, dtype: object

In [46]:
driver = webdriver.Chrome("chromedriver")
driver.maximize_window() # 창 크기 최대화

rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
user_review_id = {}
review_json = {}
image_json = {}

In [47]:
count=0
for item in items:
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=89.0.4389.90)


In [48]:
driver = webdriver.Chrome("chromedriver")
driver.maximize_window() # 창 크기 최대화

item = "강남역 스시메이진"
driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
searchbox.send_keys(item)
searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
driver.execute_script("arguments[0].click();", searchbutton)
time.sleep(2)
driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
content_url = reviewbutton.get_attribute("href") 
tabs = driver.window_handles
driver.switch_to_window(tabs[1]) # 새 탭으로 이동
driver.get(content_url) # 링크 열기



driver.switch_to_window(tabs[0])
print("기본 페이지로 돌아가자")

C:\Users\zhqhd\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: use driver.switch_to.window instead
  from ipykernel import kernelapp as app


In [2]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])

In [3]:
user_review_id = {}
review_json = {}
image_json = {}
item_meta = {}

In [5]:
df = pd.read_csv("C:/Users/정예림/Desktop/추천시스템 프로젝트/data/구별/nowon.csv", encoding="UTF-8-sig")
df=df[["상호명", "상권업종중분류명", "상권업종소분류명", "시군구명", "법정동명", "도로명주소", "kakao_keyword", "kakao_url"]]
df

,상호명,상권업종중분류명,상권업종소분류명,시군구명,법정동명,도로명주소,kakao_keyword,kakao_url
0,놀부부대찌개철판구이성북역점,한식,부대찌개/섞어찌개,노원구,월계동,서울특별시 노원구 석계로 101,월계동 놀부부대찌개철판구이성북역점,https://place.map.kakao.com/19197518
1,오딧세이월계점,유흥주점,호프/맥주,노원구,월계동,서울특별시 노원구 월계로 366,월계동 오딧세이월계점,NaN
2,서울과학기술대학교제2학생식당,기타음식업,구내식당/자급식음식점,노원구,공릉동,서울특별시 노원구 공릉로 232,공릉동 서울과학기술대학교제2학생식당,NaN
3,동대문닭한마리칼국수,한식,한식/백반/한정식,노원구,공릉동,서울특별시 노원구 공릉로38길 8-9,공릉동 동대문닭한마리칼국수,https://place.map.kakao.com/15823847
4,커피안바우,커피점/카페,커피전문점/카페/다방,노원구,월계동,서울특별시 노원구 덕릉로60길 252,월계동 커피안바우,NaN
...,...,...,...,...,...,...,...,...
4246,범피버거,패스트푸드,패스트푸드,노원구,공릉동,서울특별시 노원구 공릉로41길 12,공릉동 범피버거,https://place.map.kakao.com/2038251139
4247,휴랩,분식,라면김밥분식,노원구,공릉동,서울특별시 노원구 동일로173가길 140,공릉동 휴랩,NaN
4248,식껍,한식,갈비/삼겹살,노원구,중계동,서울특별시 노원구 한글비석로20길 38,중계동 식껍,https://place.map.kakao.com/1763761834
4249,기찻길풍경,한식,곱창/양구이전문,노원구,공릉동,서울특별시 노원구 공릉로39길 21,공릉동 기찻길풍경,https://place.map.kakao.com/1137642618


In [6]:
# kakao_url 이 없는 것 제외
nowon = df.dropna(axis=0)
nowon

,상호명,상권업종중분류명,상권업종소분류명,시군구명,법정동명,도로명주소,kakao_keyword,kakao_url
0,놀부부대찌개철판구이성북역점,한식,부대찌개/섞어찌개,노원구,월계동,서울특별시 노원구 석계로 101,월계동 놀부부대찌개철판구이성북역점,https://place.map.kakao.com/19197518
3,동대문닭한마리칼국수,한식,한식/백반/한정식,노원구,공릉동,서울특별시 노원구 공릉로38길 8-9,공릉동 동대문닭한마리칼국수,https://place.map.kakao.com/15823847
6,배스킨라빈스공릉점,패스트푸드,아이스크림판매,노원구,공릉동,서울특별시 노원구 공릉로 198,공릉동 배스킨라빈스공릉점,https://place.map.kakao.com/19153143
7,강고집즉석바지락칼국수,분식,국수/만두/칼국수,노원구,공릉동,서울특별시 노원구 공릉로38길 8-9,공릉동 강고집즉석바지락칼국수,https://place.map.kakao.com/20741840
9,아자카야,한식,한식/백반/한정식,노원구,중계동,서울특별시 노원구 덕릉로83길 20-10,중계동 아자카야,https://place.map.kakao.com/1483918776
...,...,...,...,...,...,...,...,...
4245,꼬꼬방집으로삼겹,한식,갈비/삼겹살,노원구,상계동,서울특별시 노원구 상계로17길 11,상계동 꼬꼬방집으로삼겹,https://place.map.kakao.com/1357999307
4246,범피버거,패스트푸드,패스트푸드,노원구,공릉동,서울특별시 노원구 공릉로41길 12,공릉동 범피버거,https://place.map.kakao.com/2038251139
4248,식껍,한식,갈비/삼겹살,노원구,중계동,서울특별시 노원구 한글비석로20길 38,중계동 식껍,https://place.map.kakao.com/1763761834
4249,기찻길풍경,한식,곱창/양구이전문,노원구,공릉동,서울특별시 노원구 공릉로39길 21,공릉동 기찻길풍경,https://place.map.kakao.com/1137642618


In [7]:
item = nowon[:5]
item

,상호명,상권업종중분류명,상권업종소분류명,시군구명,법정동명,도로명주소,kakao_keyword,kakao_url
0,놀부부대찌개철판구이성북역점,한식,부대찌개/섞어찌개,노원구,월계동,서울특별시 노원구 석계로 101,월계동 놀부부대찌개철판구이성북역점,https://place.map.kakao.com/19197518
3,동대문닭한마리칼국수,한식,한식/백반/한정식,노원구,공릉동,서울특별시 노원구 공릉로38길 8-9,공릉동 동대문닭한마리칼국수,https://place.map.kakao.com/15823847
6,배스킨라빈스공릉점,패스트푸드,아이스크림판매,노원구,공릉동,서울특별시 노원구 공릉로 198,공릉동 배스킨라빈스공릉점,https://place.map.kakao.com/19153143
7,강고집즉석바지락칼국수,분식,국수/만두/칼국수,노원구,공릉동,서울특별시 노원구 공릉로38길 8-9,공릉동 강고집즉석바지락칼국수,https://place.map.kakao.com/20741840
9,아자카야,한식,한식/백반/한정식,노원구,중계동,서울특별시 노원구 덕릉로83길 20-10,중계동 아자카야,https://place.map.kakao.com/1483918776


In [8]:
for i, row in item.iterrows() :
    url = item["kakao_url"][i]
    print(i, url,item["상호명"][i])

0 https://place.map.kakao.com/19197518 놀부부대찌개철판구이성북역점
3 https://place.map.kakao.com/15823847 동대문닭한마리칼국수
6 https://place.map.kakao.com/19153143 배스킨라빈스공릉점
7 https://place.map.kakao.com/20741840 강고집즉석바지락칼국수
9 https://place.map.kakao.com/1483918776 아자카야


In [43]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path='chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 

In [44]:
for i, row in item.iterrows() :
    
    url = item["kakao_url"][i]
    print(i,"번째 크롤링 중 : ",item["상호명"][i] , sep=" ")
    
    # 상세보기 접속
    driver.get(url)
    
    # html  파싱
    html = driver.page_source
    time.sleep(3)
    soup = BeautifulSoup(html, 'lxml')
    time.sleep(3)
    
    # 리뷰 갯수 --> 몇 페이지인지 알기위해 
    try :
        driver.find_element_by_xpath('//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[2]').click()
        rating_num = driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_sorting > a > span.color_b').text
        rating_num = int(rating_num)
        print(rating_num)
        
        # 페이지 수 --> 리뷰 수가 5개 이하이면 한 페이지, 6개 이상이면 n개의 페이지
    
        if rating_num > 0 and rating_num <=5 :
            page_num = 1
            print(page_num)
            
            for j in range(1,page_num) :
                # 해당 페이지의 리뷰리스트 가져오기
                # 리뷰 리스트는 list_evaluation 클래스 안에 li 태그에 있음
                review_lists = driver.find_elements_by_css_selector("#mArticle > div.cont_evaluation > div.evaluation_review > ul > li")
        
                #1-5 페이지
                if j <= 4 :
                    #리뷰 리스트가 0이 아니면
                    if len(review_lists) != 0 :
                        for m, review in enumerate(review_lists) : ## --->  데이터 긁어오는거 안됨... 해결해랴!!!
                            data=[]
                            UserID = soup.select('.append_item > a[data-userid]')[0].get('data-userid') #UserID
                            ItemID = item["상호명"][i]                                                  #ItemID
                            Rating = int(soup.select('.grade_star > em')[0].text)                       #Rating
                            Timestamp = soup.select('.time_write')[0].text                              #Timestamp
                            data = UserID, ItemID, Rating, Timestamp      
                            print(data)
                            rating_df.append(data)                                                      #UserID, ItemID, Rating, Timestamp를 rating_df에 append

                        try :
                            # 다음 페이지로 넘어가기
                            driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child('+str(j+2)+')').click()
                            time.sleep(2)
                        except (NoSuchElementException, ElementNotInteractableException) :
                            continue

                elif j == 5:
                    #리뷰 리스트가 0이 아니면
                    if len(review_lists) != 0 :
                        for m, review in enumerate(review_lists) :
                            data=[]
                            UserID = soup.select('.append_item > a[data-userid]')[0].get('data-userid') #UserID
                            ItemID = item["상호명"][i]                                                  #ItemID
                            Rating = int(soup.select('.grade_star > em')[0].text)                       #Rating
                            Timestamp = soup.select('.time_write')[0].text                              #Timestamp
                            data = UserID, ItemID, Rating, Timestamp
                            print(data)
                            rating_df.append(data)                                                      #UserID, ItemID, Rating, Timestamp를 rating_df에 append

                    try :
                        # 다음 페이지로 넘어가기
                        driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a.btn_next') #다음 버튼 누르기
                        time.sleep(2)
                    except (NoSuchElementException, ElementNotInteractableException) :
                        continue
                    
        else :
            page_num = int(math.ceil(rating_num / 5))
            print(page_num)
            
            for j in range(1,page_num) :
                # 해당 페이지의 리뷰리스트 가져오기
                # 리뷰 리스트는 list_evaluation 클래스 안에 li 태그에 있음
                review_lists = driver.find_elements_by_css_selector("#mArticle > div.cont_evaluation > div.evaluation_review > ul > li")

                #1-5 페이지
                if j <= 4 :
                    #리뷰 리스트가 0이 아니면
                    if len(review_lists) != 0 :
                        for m, review in enumerate(review_lists) :
                            data=[]
                            UserID = soup.select('.append_item > a[data-userid]')[0].get('data-userid') #UserID
                            ItemID = item["상호명"][i]                                                  #ItemID
                            Rating = int(soup.select('.grade_star > em')[0].text)                       #Rating
                            Timestamp = soup.select('.time_write')[0].text                              #Timestamp
                            data = UserID, ItemID, Rating, Timestamp      
                            print(data)
                            rating_df.append(data)                                                      #UserID, ItemID, Rating, Timestamp를 rating_df에 append

                        try :
                            # 다음 페이지로 넘어가기
                            driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child('+str(j+2)+')').click()
                            time.sleep(2)
                        except (NoSuchElementException, ElementNotInteractableException) :
                            continue
        
                elif j == 5:
                    #리뷰 리스트가 0이 아니면
                    if len(review_lists) != 0 :
                        for m, review in enumerate(review_lists) :
                            data=[]
                            UserID = soup.select('.append_item > a[data-userid]')[0].get('data-userid') #UserID
                            ItemID = item["상호명"][i]                                                  #ItemID
                            Rating = int(soup.select('.grade_star > em')[0].text)                       #Rating
                            Timestamp = soup.select('.time_write')[0].text                              #Timestamp
                            data = UserID, ItemID, Rating, Timestamp
                            print(data)
                            rating_df.append(data)                                                      #UserID, ItemID, Rating, Timestamp를 rating_df에 append

                    try :
                        # 다음 페이지로 넘어가기
                        driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a.btn_next') #다음 버튼 누르기
                        time.sleep(2)
                    except (NoSuchElementException, ElementNotInteractableException) :
                        continue
    
        
                 #6페이지 이상
                elif j >= 6 :
            
                    #리뷰 리스트가 0이 아니면
                    if len(review_lists) != 0 :
                        for m, review in enumerate(review_lists) :
                            data=[]
                            UserID = soup.select('.append_item > a[data-userid]')[0].get('data-userid')
                            ItemID = item["상호명"][i]
                            Rating = int(soup.select('.grade_star > em')[0].text)
                            Timestamp = soup.select('.time_write')[0].text
                            data = UserID, ItemID, Rating, Timestamp
                            print(data)
                            rating_df.append(data)


                        # 다음 페이지로 넘어가기
                        try :
                            if (j%5) == 1 :
                                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(4)').click()
                                time.sleep(2)
                            elif (j%5) == 2 :
                                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(5)').click()
                                time.sleep(2)
                            elif (j%5) == 3 :
                                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(6)').click()
                                time.sleep(2)
                            elif (j%5) == 4 :
                                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(7)').click()
                                time.sleep(2)
                            else :
                                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a.btn_next').click()
                                time.sleep(2)
                        except (NoSuchElementException, ElementNotInteractableException) :
                            continue
    
            else : 
                print("리뷰가 없습니다.")
        
    except (NoSuchElementException, ElementNotInteractableException) :
        print("리뷰가 없습니다")
        continue 
    
    
    

0 번째 크롤링 중 :  놀부부대찌개철판구이성북역점
3
1
3 번째 크롤링 중 :  동대문닭한마리칼국수
리뷰가 없습니다
6 번째 크롤링 중 :  배스킨라빈스공릉점
6
2


IndexError: list index out of range

In [40]:
rating_df

,UserID,ItemID,Rating,Timestamp
